In [1]:
from fpl import (
    Loader,
    Player,
    Team,
    SimpleExpectedPointsCalculator,
    Optimizer,
    find_matching_players,
)
import heapq

## Dowloading your personal FPL team

### `Loader.get_my_team(login, password, manager_id, how, filename)`

To download from the API you need your login details and `manager_id`. To get your `manager_id`:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/) and navigate to the points tab.
2. Your `manager_id` should appear somewhere in the URL. [FPL APIs Explained](https://www.oliverlooney.com/blogs/FPL-APIs-Explained#how-to-use-authenticated-endpoints) has some screenshots of this.

If the download from the API fails, you can manually download the json blob as a local file:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/)
2. View a json blob of your team by typing this URL `https://fantasy.premierleague.com/api/my-team/{manager_id}/` into your browser with the appropriate `manager_id`
3. Save the blob locally for example as `my_team.json`


In [2]:
# team = Loader.get_my_team(login="email", password="password", manager_id=123456, how="api")
team = Loader.get_my_team(
    login="", password="", manager_id=0, how="local", filename="resources/my_team.json"
)
print(team)

---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=585

## Creating a candidate list

### `find_matching_players(search_name, matching_threshold)`
This function helps you create a `candidates` list of players you want to transfer in. Remember to search by a player's name as it appears on [Fantasy Premier League](https://fantasy.premierleague.com/).

In [3]:
find_matching_players(search_name="alexander arnold", threshold=50)

[(Player(element=115, name='Baleba', position=3, club=5, cost=50),
  'Carlos Baleba'),
 (Player(element=223, name='Garner', position=3, club=8, cost=49),
  'James Garner'),
 (Player(element=240, name='Andreas', position=3, club=9, cost=49),
  'Andreas Hoelgebaum Pereira'),
 (Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=74),
  'Trent Alexander-Arnold'),
 (Player(element=642, name='André', position=3, club=20, cost=50),
  'André Trindade da Costa Neto')]

In [4]:
candidates = [
    Player(element=398, name="Gordon", position=3, club=15, cost=77),
    Player(element=339, name="Virgil", position=2, club=12, cost=64),
    Player(element=354, name="Kovačić", position=3, club=13, cost=54),
    Player(element=321, name="Gakpo", position=4, club=12, cost=76),
    Player(element=364, name="Amad", position=3, club=14, cost=56),
]

## Optimizing your team

The optimization functions require a parameter `epc`, which should be a class inheriting from the abstract class `ExpectedPointsCalculator`. This class provides a prediction of how many points a player will score in a particular gameweek via a single function:
- `get_expected_points(player_id, gameweek)`: Returns the expected points for a player in a specific gameweek.

You can easily integrate your own predictive models by creating classes that inherit from `ExpectedPointsCalculator`.

In the following simple examples, we use a `SimpleExpectedPointsCalculator`. Later, we show how to define your own expected points calculator by inheriting from `ExpectedPointsCalculator`.

### `Optimizer.calc_discounted_reward_player(player, epc, gameweek, horizon, gamma)`
Calculate how you think each player will perform on an individual basis over the coming gameweeks.

In [5]:
{
    c: Optimizer.calc_discounted_reward_player(
        player=c,  # A player.
        epc=SimpleExpectedPointsCalculator,  # How you want to calculate the expected points for each player.
        gameweek=Loader.get_next_gameweek(),  # Which gameweek do you want to start on.
        horizon=5,  # How many gameweeks do you want to calculate over.
        gamma=1,  # The discount factor.
    )
    for c in candidates
}

{Player(element=398, name='Gordon', position=3, club=15, cost=77): 18.700000000000003,
 Player(element=339, name='Virgil', position=2, club=12, cost=64): 18.3,
 Player(element=354, name='Kovačić', position=3, club=13, cost=54): 17.625,
 Player(element=321, name='Gakpo', position=4, club=12, cost=76): 19.9,
 Player(element=364, name='Amad', position=3, club=14, cost=56): 0.0}

### `Optimizer.calc_optimial_formation(team, epc, gameweek)`
Calculate the best possible formation for a particular gameweek - the formation is filled with players having the greatest expected points.

In [6]:
print(
    Optimizer.calc_optimal_formation(
        team=team,  # Some team.
        epc=SimpleExpectedPointsCalculator,  # How you want to calculate the expected points for each player.
        gameweek=Loader.get_next_gameweek(),  # Which gameweek do you want to start on.
    )
)

---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
---------------------------------------------------------------------------
{FWDS}
Player(element=129, name='João Pedro', position=4, club=5, cost=55)
Player(element=401, name='Isak', position=4, club=15, cost=92)


### `Optimizer.calc_discounted_reward_team(team, epc, gameweek, horizon, gamma, wildcard)`
Calculate how you think a particular team will perform over the coming gameweeks. Each gameweek the formation will be optimized to play the best possible team. A penalty will be applied depending on how many free transfers you have.

In [7]:
new_team = team.transfer_player(
    out_player=Player(element=328, name="M.Salah", position=3, club=12, cost=132),
    in_player=Player(element=240, name="Andreas", position=3, club=9, cost=50),
)

In [8]:
Optimizer.calc_discounted_reward_team(
    team=new_team,  # Some team.
    epc=SimpleExpectedPointsCalculator,  # How you want to calculate the expected points for each player.
    gameweek=Loader.get_next_gameweek(),  # Which gameweek do you want to start on.
    horizon=1,  # How many gameweeks do you want to calculate over.
    gamma=1.0,  # The discount factor.
    wildcard=True,  # Flag when True turns off the transfer adjustment.
)

68.10000000000001

### `Optimizer.calc_optimal_team(team, candidates, epc, gameweek, horizon, max_transfers, gamma, wildcard)`
Calculate the top three possible teams given a list of candidates. This will create all possible teams with `0` to `max_transfers` transfers and compute the discounted reward of each team.

In [9]:
top_three = Optimizer.calc_optimal_teams(
    team=team,  # Your current fpl team.
    candidates=candidates,  # Whichever players you want to transfer in.
    epc=SimpleExpectedPointsCalculator,  # How you want to calculate the expected points for each player.
    gameweek=Loader.get_next_gameweek(),  # Which gameweek do you want to start on.
    horizon=5,  # How many gameweeks do you want to calculate over.
    max_transfers=3,  # The maximum number of transfers allowed in the optimization.
    gamma=0.8,  # The discount factor.
    wildcard=False,  # Flag when True turns off the transfer adjustment.
)
third_best_score, third_best_team = heapq.heappop(top_three)
second_best_score, second_best_team = heapq.heappop(top_three)
first_best_score, first_best_team = heapq.heappop(top_three)
print("Score: {}".format(first_best_score))
print(first_best_team)

Score: 250.49588
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
P

## Creating your own `ExpectedPointsCalculator`

To create your own expected points calculator, inherit from `ExpectedPointsCalculator` and implement the `get_expected_points` method.